# Házi feladat

Az $S=S(X)=(S_k)_{k=0}^n$ sorozatot rekurzióval definiáljuk az
$(X_k)_{k=1}^n$ előjel sorozatból az
$$
  S_0=0,\quad S_k=S_{k-1}+X_k, \quad 0 < k \leq n
$$
formulával.

$S_k$-ra úgy gondolunk, mint egy bolyongó részecske helyzetére a $k$.
lépés után, ha az $X$ előjel sorozat adja meg a lépéseket.

Legyen $T(X)$ az $X$ előjel sorozat következő permutációja: először
azokat az $X_k$ lépéseket soroljuk fel melyekre $S_k > 0$ fordított
sorrendben, majd ezek után írjuk azokat az $X_k$ lépéseket, ahol 
$S_k\leq 0$ megőrizve az eredeti sorrendjüket.


- Írjunk egy függvényt, ami adott előjel sorozatra kiszámolja a képét a $T$ leképezésnél.

- Az első néhány $n$-re ($n=1,\dots,16$) ellenőrizzük le, hogy
  $T:\left\{{-1,1}\right\}^n\to\left\{{-1,1}\right\}^n$ bijekció.

- Az előző pont alapján azt sejthetjük, hogy $T$ tetszőleges $n$
  esetén a $\left\{{-1,1}\right\}^n$ halmaz permutációja. Írjuk meg a $T$
  inverzét kiszámító függvényt.
- Készítsünk egy ábrát a következő módon. Legyen $n$ nem túl
  kicsi, mondjuk 100 és 10000 közti. Sorsoljunk ki egy $n$ hosszú $X$
  előjel sorozatot. Ábrázoljuk az $S(X)$ bolyongást.

  Ezután készítsük el a $T(X)$-hez tartozó bolyongás ábráját.

- Az első néhány $n$-re ($n=1,\dots,16$) ellenőrizzük le, hogy
  $$
    \text{maxhely}(S(T(X))) = \text{pozitívidő}(S(X))
  $$
  ahol
  $$
    \text{maxhely}(s) = \min\left\{{k}\,:\,{s_k =\max s_j}\right\} 
    \quad\text{és}\quad
    \text{pozitívidő}(s) = \sum_k \mathbb{1}_{s_k>0}
  $$  

Extra 3 pontot ér annak a formális bizonyítása, hogy
$$
\text{maxhely}(S(T(X))) = \text{pozitívidő}(S(X))
$$



In [ ]:
from collections import deque


def T(x):
    result = deque(maxlen=len(x))
    s = 0
    for step in x:
        s += step
        if s > 0:
            result.appendleft(step)
        else:
            result.append(step)
    return [*result]


In [ ]:
import matplotlib.pyplot as plt
from itertools import accumulate
import numpy as np

In [ ]:
steps = [1, 1, -1, -1, 1]

walk = np.array([*accumulate(steps, initial=0)])

plt.plot(walk, label="S")
plt.grid()

In [ ]:
print(f"{steps=}, {T(steps)=}")

In [ ]:
for n in range(1, 17):
    range_t = set()
    for num in range(1 << n):
        steps = [1 if num & (1 << i) else -1 for i in range(n)]
        range_t.add(tuple(T(steps)))
    assert len(range_t) == 1 << n

In [ ]:

def Tinv(x):
    s = sum(x)
    x = deque(x)
    result = []
    while x:
        step = x.popleft() if s > 0 else x.pop()
        s -= step
        result.append(step)
    result.reverse()
    return result


In [ ]:
Tinv(T(steps)) == steps

In [ ]:
for n in range(1, 16):
    for num in range(1<<n):
        steps = [2*((num >> i) & 1)-1 for i in range(n)]
        assert Tinv(T(steps)) == steps


In [ ]:
def walk(steps):
    return np.array([*accumulate(steps, initial=0)])

def arg_max(steps):
    w = walk(steps)
    return (w==w.max()).nonzero()[0].min()

def pos_time(steps):
    w = walk(steps)
    return (w > 0).sum()
    


In [ ]:
steps = np.where(np.random.binomial(1, 0.5, 1000), 1, -1)
plt.plot(walk(steps), label="S", alpha=0.5)
plt.plot(walk(T(steps)), label="T(S)", alpha=0.5)
plt.legend()
plt.grid()

In [ ]:
steps = [1, 1, -1, -1, 1]
plt.plot(walk(steps), "bo-")
print(f"{arg_max(steps)=}, {pos_time(steps)=}")

In [ ]:
for n in range(1, 16):
    for num in range(1<<n):
        steps = [2*((num >> i) & 1)-1 for i in range(n)]
        assert arg_max(T(steps)) == pos_time(steps)


**Bizonyítás.**

Ötlet: bolyongás (lépések) felbontása _kirándulásokra_.
$$
\sigma_0 = 0, \quad 
\sigma_{2k+1} = \inf\{n\geq 
\sigma_{2k}\colon S_n = 1\}, \quad 
\sigma_{2k} = \inf\{n\geq 
\sigma_{2k-1}\colon S_n = 0\}
$$

Egy kirándulás a pozitív oldalon a $\sigma_{2k+1}$. lépéssel kezdődik és a $\sigma_{2k+2}$. lépésig tart, az utolsó lépés már nem tartozik hozzá. 

Egy kirándulás a negatív oldalon  a $\sigma_{2k}$. lépéssel kezdődik és a $\sigma_{2k+1}$. lépésig tart, az utolsó lépés már nem tartozik hozzá. 

A pozitív oldalon töltött idő $n$ lépés alatt:
$$
    \sum_{k=1}^\infty (\sigma_{2k}\wedge n)-(\sigma_{2k-1}\wedge n)
$$

Ha az $(X_k)_{\sigma_{2k-1}\leq k < \sigma_{2k}}$ lépéseket fordítva soroljuk fel, akkor egy olyan lépéssorozatot **fordítunk meg**, ami a nulla szintről az egy szintre jut és közben végig az a nulla szint felett halad. 

A lépések fordított felsorolása a középpontos tükrözésnek felel meg, ezért a mikor fordított sorrendben soroljuk fel az egy pozitív kiránduláshoz tartozó lépéseket, akkor olyan bolyngást kapunk, ami egységnyit emelkedik és végig az utolsó érték alatt marad. Ez mutatja, hogy a $T(X)$ sorozatból számolt bolyongás a $\text{pozitív idő}(S(X))$ időpontban nagyobb mint ezek előtt.

A negatív kirándulásokra hasonló érvelés mutatja, hogy ha egy negatív kirándulás egy pozitív kirándulás után jön akkor az első lépése lefelé visz, utána végig a nulla szint alatt marad. Ha pedig a nulla időpontban kezdődik, akkor nullából indul és esetleg többször is visszatérhet a 0-ba, de efelé már nem mehet, mert az a lépés már a következő pozitív kiránduláshoz van sorolva. Így, amikor $\text{pozitív idő}(S(X))$ időpont után a negatív kirándulásokat kezdjük el felhasználni, még néhányszor visszatérhet a bolyongás a $\text{pozitív idő}(S(X))$ időpontban felvett értékhez, de afölé már nem mehet.

In [ ]:
def excursions(steps):
    w = walk(steps)
    sigma = 0
    next_goal = 1 if steps[0] == -1 else 0
    pos_excursions = []
    neg_excursions = []
    excursion = []
    while sigma < len(steps):
        if w[sigma+1] == next_goal:
            if next_goal == 1:
                neg_excursions.append(excursion)
            else:
                pos_excursions.append(excursion)
            next_goal = 1-next_goal
            excursion = []
        excursion.append(steps[sigma])        
        sigma += 1
    if next_goal == 1:
       neg_excursions.append(excursion)
    else:
        pos_excursions.append(excursion)
    return pos_excursions, neg_excursions

In [ ]:
steps = [1, 1, -1, -1, -1, 1, 1, 1]
print(f"{walk(steps)=}, {excursions(steps)=}")

In [ ]:
steps = np.where(np.random.binomial(1, 0.5, 1000), 1, -1)
pos_excursions, neg_excursions = excursions(steps)

In [ ]:
if len(pos_excursions) > 0:
    fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(8, 3))
    ax0.plot(walk(pos_excursions[0]), "r-")
    ax1.plot(walk(reversed(pos_excursions[0])), "b-")
    ax0.grid(); ax1.grid()
    plt.show()
    fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(8, 3))
    ax0.plot(walk(pos_excursions[-1]), "r-")
    ax1.plot(walk(reversed(pos_excursions[-1])), "b-")
    ax0.grid(); ax1.grid()
    plt.show()
    


In [ ]:
if len(neg_excursions) > 0:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4, 3))
    ax.plot(walk(neg_excursions[0]), "r-")
    ax.grid()
    plt.show()
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4, 3))
    ax.plot(walk(neg_excursions[-1]), "r-")
    ax.grid()
    plt.show()


# Conway Game of Life

[Wikipédia](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life)

The Game of Life is a cellular automaton created by mathematician John Conway in 1970. The game consists of a board of cells that are either on or off. One creates an initial configuration of these on/off states and observes how it evolves. There are four simple rules to determine the next state of the game board, given the current state:

- **Overpopulation**: if a living cell is surrounded by more than three living cells, it dies.
- **Stasis**: if a living cell is surrounded by two or three living cells, it survives.
- **Underpopulation**: if a living cell is surrounded by fewer than two living cells, it dies.
- **Reproduction**: if a dead cell is surrounded by exactly three cells, it becomes a live cell.

****

Írjunk egy osztályt a játékhoz, pl. az `__init__` metódus hozza létre a megadott konfigurációnak megfelelő objektumot. Legyen egy `step` metódus, ami a rendszert a következő állapotába viszi és az `__str__` metódus pedig
valahogy ábrázolja az aktuális állapotot.

Tegyük fel, hogy a rács amin a rendszer él, egy $m\times n$-es rács, ahol mindkét irányban ciklikusan körbemegyünk,
azaz a csúcsokat modulo $m$ ill. modulo $n$ tekintjük.


Alternatíva lehetne, hogy ha új elő sejt keletkezik az aktuális rács körül, akkor megnöveljük a rácsunkat az adott irányban.  

In [ ]:
import numpy as np

class ConwayGoL:

    def __init__(self, state):
        self.state = np.asarray(state)

    def step(self):
        return self

    def __repr__(self):
        return f"{type(self).__name__}({self.state})"

In [ ]:


init_state = [] ## ???
conway = ConwayGoL(init_state)

conway.step()


A játék állapotának leírásához egy $m\times n$ rács minden pontjáról tudni kell, hogy foglalt-e vagy sem.

```python
m, n = 11, 11
state = np.zeros((11, 11), dtype=np.uint8)
```

Véletlenszerű kezdeti állapot:
```python
state = np.random.binomial(1, p, size=(m, n))
```

In [ ]:
def random_state(n, m, p):
    return np.random.binomial(1, p, (n, m)).astype(np.uint8)

state = random_state(5, 6, 0.2)
print(state)

Szebb megjelenítés?

In [ ]:
# def as_matrix(lst, n):
#     return [lst[i:i+n] for i in range(0, len(lst), n)]

print('\n'.join(''.join(map(str, line)) for line in state))


In [ ]:
for symbols in [
    "\u2b1c\u2b1b",
    "·♥",
    "🟡🟥"
    ]:
    print('\n'.join(''.join(symbols[x] for x in line) for line in state))


In [ ]:
import matplotlib.pyplot as plt

img = plt.matshow(
    state, 
    cmap="Pastel1_r", 
    vmax=1, vmin=0, alpha=0.8)
img.axes.axis("off")
n, m = len(state), len(state[0])
for pos in range(0, n+1):
    img.axes.axhline(y=pos-0.5, color="gray")
for pos in range(0, m+1):
    img.axes.axvline(x=pos-0.5, color="gray")

plt.show()


In [ ]:

def cgol_str(self):
    symbols = "\u2b1c\u2b1b" # ⬜⬛
    return '\n'.join(''.join(symbols[x] for x in line) for line in self.state)

ConwayGoL.__str__ = cgol_str



In [ ]:
conway = ConwayGoL(state)
print(conway)

A `step` metódushoz ki kellene számolni egy adott csúcs foglalt szomszédainak számát `cnt`. Ha ez kész,
akkor az $i$ csúcs új állapota:

$$
    \text{state}_{t+1}[i]=
    \begin{cases}
    1 &\text{Ha $\text{cnt}[i]\in\{2,3\}$ és $\text{state}_t[i]=1$}\\
    1 &\text{Ha $\text{cnt}[i]\in\{3\}$ és $\text{state}_t[i]=0$}\\
    0 &\text{különben}
    \end{cases}
$$

In [ ]:
def newstate(state, count):
    return (count==3) | (state & (count==2))
    # return [int((c==3)|((c==2) & (s==1))) for s, c in  zip(state, count)]

In [ ]:
import ipytest
ipytest.autoconfig()

In [ ]:
%%ipytest

def test_newstate():
    res = np.zeros(9, dtype=np.uint8)
    res[2] = 1
    res[3] = 1
    assert (newstate(np.ones(9, dtype=np.uint8), np.arange(9, dtype=np.uint8)) == res).all()
    res = np.zeros(9, dtype=np.uint8)
    res[3] = 1
    assert (newstate(np.zeros(9, dtype=np.uint8), np.arange(9,dtype=np.uint8)) == res).all()


In [ ]:
def count_neighbors(state):
    h, w = state.shape
    count = np.zeros((h+2, w+2), dtype=state.dtype)
    for dh, dw  in [(0,-1), (0, 1), (1,-1), (1,0), (1,1), (-1,-1), (-1,0), (-1,1)]:
        count[1+dh:1+dh+h, 1+dw:1+dw+w] += state
    return count 

def cgol_step(self):
    counts = count_neighbors(self.state)
    state = newstate(np.pad(self.state, 1), counts)
    if state[0].max() == 0:
        state = state[1:]
    if state[-1].max() == 0:
        state = state[:-1]
    if state[:,0].max() == 0:
        state = state[:,1:]
    if state[:,-1].max() == 0:
        state = state[:,:-1]
    self.state = state
    return self

ConwayGoL.step = cgol_step

In [ ]:
@classmethod
def cgol_from_random_state(cls, n, m,  p):
    return cls(random_state(n, m, p))

ConwayGoL.from_random_state=cgol_from_random_state

In [ ]:
conway = ConwayGoL.from_random_state(5, 10, 0.2)
print(conway)
print(*count_neighbors(conway.state), sep='\n')

In [ ]:
conway = ConwayGoL.from_random_state(5, 5, 0.25)
print(conway)
print("-"*20)
print(conway.step())

Tudunk-e valami animációszerűt készíteni? Jupyter notebook-ban pl. a következő képpen lehet:

In [ ]:
from ipywidgets import Output
from time import sleep


In [ ]:

out = Output()
display(out)
conway = ConwayGoL.from_random_state(n=21, m=51, p=0.2)

for i in range(50):
    out.clear_output(True)
    with out:
        print(f"After {i} steps:\n{conway}")
    sleep(0.15)
    conway.step()


## kis ellenőrzés

In [ ]:
state = random_state(11, 21, 0.2)
plt.matshow(np.pad(state, 1), cmap='Pastel1_r', vmax=1, vmin=0)
plt.xticks(np.arange(state.shape[1]+2)-.5, minor=True)
plt.xticks([])
plt.yticks(np.arange(state.shape[0]+2)-.5, minor=True)
plt.yticks([])
plt.grid(which="minor", color="gray", linestyle='-', linewidth=1)
plt.axis()

for (i, j), cnt in np.ndenumerate(count_neighbors(state)):
    plt.text(j, i, str(cnt), ha="center", va="center")


In [ ]:
plt.matshow(np.pad(state, 1), cmap='Pastel1_r', vmax=1, vmin=0)
plt.xticks(np.arange(state.shape[1]+2)-.5, minor=True)
plt.xticks([])
plt.yticks(np.arange(state.shape[0]+2)-.5, minor=True)
plt.yticks([])
plt.grid(which="minor", color="gray", linestyle='-', linewidth=1)
plt.axis()

for (i, j), cnt in np.ndenumerate(newstate(np.pad(state, 1), count_neighbors(state))):
    plt.text(j, i, str(cnt), ha="center", va="center")


In [ ]:
out = Output()
display(out)
state = random_state(n=51, m=21, p=0.2)

for i in range(50):
    out.clear_output(True)
    with out:
        print(f"After {i} steps:\n{str_state(state)}")
    sleep(0.15)
    state = new_state(state)


## Parancssoros script

Ha parancssorból dolgozunk, akkor valami ilyesmit lehetne tenni

In [ ]:
%%writefile conway.py

import numpy as np

def random_state(n, m, p):
    return np.random.binomial(1, p, (n, m)).astype(np.uint8)


def count_neighbors(state):
    h, w = state.shape
    count = np.zeros((h+2, w+2), dtype=state.dtype)
    for dh, dw  in [(0,-1), (0, 1), (1,-1), (1,0), (1,1), (-1,-1), (-1,0), (-1,1)]:
        count[1+dh:1+dh+h, 1+dw:1+dw+w] += state
    return count     


def newstate(state, count):
    return ((count==3) | ((count==2) & (state==1))).astype(state.dtype)


class ConwayGoL:
    symbols = "\u2b1c\u2b1b"

    def __init__(self, state):
        self.state = np.asarray(state)

    def step(self):
        counts = count_neighbors(self.state)
        state = newstate(np.pad(self.state, 1), counts)
        if state[0].max() == 0:
            state = state[1:]
        if state[-1].max() == 0:
            state = state[:-1]
        if state[:,0].max() == 0:
            state = state[:,1:]
        if state[:,-1].max() == 0:
            state = state[:,:-1]
        self.state = state
        return self

    def __str__(self):
        symbols = self.symbols
        return '\n'.join(''.join(symbols[x] for x in line) for line in self.state)

    def __repr__(self):
        return f"{type(self).__name__}({self.state})"

    @classmethod
    def from_random_state(cls, m, n, p):
        return cls(random_state(m, n, p))

    def is_empty(self):
        return self.state.max()==0


def clear_terminal(n):
    print(f"{chr(27)}[{n+1}A", end="")

def main(m=11, n=25, p=0.2, nsteps=10, clear_screen=clear_terminal):
    from time import sleep
    conway = ConwayGoL.from_random_state(m, n, p)
    for i in range(nsteps+1):
        if i>0:
            clear_screen(conway.state.shape[0])
        print(f"after {i} step:")
        print(conway)
        sleep(0.2)
        conway.step()
        if conway.is_empty():
            break

if __name__ == "__main__":
    main()


Ha valamit már megírtunk és szeretnénk használni, `import`-tal elérhető. Pl.

### Tudunk-e paramétereket adni a python scriptnek?

Amikor egy python scriptet futtatunk, a parancssor (amivel a futást indítottuk) a `sys` modul `argv` változójában érhető el.

In [ ]:
import sys
sys.argv

In [ ]:
! python -c 'import sys; print(sys.argv)' -alma


Egy nagyon egyszerű megoldás, ha minden opciónak a neve a paraméter amit beállít és egyenlőségjel után az értéke:
pl. n=11 m=25 nstep=10 p=0.2

In [ ]:
cmdline = "conway.py -n=11 -m=25 -nstep=10 -p=0.2"
argv = cmdline.split()
params =[param.split("=") for param in argv[1:]]
params

Minden paraméterről tudni kellene, hogy milyen típusú!

In [ ]:
param_types={'-n': int, '-m': int, '-nstep': int, '-p': float}
params = {k.replace("-",""): param_types[k](v)  for k, v in (param.split("=") for param in argv[1:])}
params

Ezek után a `main` függvényt a megadott paraméterekkel meg tudjuk hívni:

```
    main(**params)
```
Mi van a `default` értékekkel, `help`-pel stb.

Ezeket mind meg tudnánk írni, de nem kell. Van kész megoldás `python`-ban.

Az `argparse` könvytár mindent megcsinál, ami nekünk kell.

In [ ]:
import argparse

help(argparse)

A `conway.py` file végét cseréljük le erre.

In [ ]:
%%writefile conway_cli.py

import conway

if __name__ == "__main__":
    import argparse
    
    parser = argparse.ArgumentParser(description='Conways Game of Life')

    parser.add_argument(
        '-n', '--nrows',
        type=int,
        default=11,
        help='number of rows'
        )

    parser.add_argument(
        '-m', '--ncols',
        type=int,
        default=25,
        help='number of columns'
        )

    parser.add_argument(
        '-p', '--density',
        type=float,
        default=0.2,
        help='initial density')
    
    parser.add_argument(
        '--nsteps',
        type=int,
        default=10,
        help='steps to display'
        )

    args = parser.parse_args()
    print(args)
    
    conway.main(n=args.ncols, m=args.nrows, p=args.density, nsteps=args.nsteps)


In [ ]:
! python conway_cli.py

In [ ]:
# import importlib
# importlib.reload(conway)

In [ ]:
import conway

out1 = Output()
display(out1)

with out1:
    conway.main(clear_screen=lambda n: out1.clear_output(True))

Az `argparse` könyvtár nem a legkényelmesebb. Alternatívák:

- [Docopt](http://docopt.org/)
- [Click](https://pypi.org/project/click/)
- [clize](https://github.com/epsy/clize)

és még sok másik is!

## Itt is használhattunk volna dekorátort


A `ConwayGoL` példában utólag adtunk metódusokat az osztályunkhoz. Ezt is megtehettük volna dekorátorral.  

In [ ]:
def conway_method(f):
    setattr(ConwayGoL, f.__name__, f)
    return f


@conway_method
def dummy_method(self):
    print("this is a message from the new method!")

c = ConwayGoL([])
c.dummy_method()

Azt is megtehettük volna, hogy a osztály nincs beleégetve a kódba.

In [ ]:
def new_method(cls):
    def decorator(f):
        setattr(cls, f.__name__, f)
        return f
    return decorator

@new_method(ConwayGoL)
def dummy_method(self):
    print("Note that the old value of dummy_method is overwritten!")

In [ ]:
c.dummy_method()

## Néhány minta

In [ ]:
import urllib.request as request


In [ ]:

urls = [
    "https://conwaylife.com/patterns/83p7h1v1.cells",
    "https://conwaylife.com/patterns/rotatedhouse.cells",
]



In [ ]:
with request.urlopen(urls[0]) as file:
    btext = file.read()
text = btext.decode("utf-8")
print(text)
data = [[0 if c == "." else 1 for c in line] for line in text.splitlines() if not line.startswith("!")]


In [ ]:
from ipywidgets import Output
from time import sleep
import numpy as np


In [161]:

out = Output()
display(out)


conway_gol = conway.ConwayGoL(data)

for i in range(100):
    out.clear_output(True)
    with out:
        print(f"After {i} steps:\n{conway_gol}")
    sleep(0.15)
    conway_gol.step()
    if conway_gol.is_empty():
        break


Output()

In [163]:
with request.urlopen(urls[1]) as file:
    btext = file.read()
text = btext.decode("utf-8")
print(text)
data = [[0 if c == "." else 1 for c in line] for line in text.splitlines() if not line.startswith("!")]


! rotatedhouse.cells
! https://conwaylife.com/wiki/Rotated_house
! https://www.conwaylife.com/patterns/rotatedhouse.cells
OO.....
O.O....
..O..OO
O.O.O.O
OO..O..
....O.O
.....OO



In [165]:

out = Output()
display(out)


conway_gol = conway.ConwayGoL(data)

for i in range(10):
    out.clear_output(True)
    with out:
        print(f"After {i} steps:\n{conway_gol}")
    sleep(0.15)
    conway_gol.step()
    if conway_gol.is_empty():
        break


Output()

# Egy gráfelméleti algoritmus

## Feladat

Adott egy `n` csúcsú irányítatlan gráf az élek listájával. A gráf csúcsait `0`-tól `n-1`-ig címkéztük meg, az éleket pedig a végpontokkal.

Emellett adott egy kiindulási pont és egy végpont. Azt szeretnénk eldönteni, hogy el lehet-e jutni a kiindulási pontból a végpontba a gráf éleit használva.

Gondolhatunk arra, hogy a gráf egy úthálózatot ír le és a kérdés az, hogy el tudunk-e jutni `A`-ból `B`-be.

Pl. `n = 3`, élek `edges = [[0,1], [1,2], [2,0]]`, `A =  0`, `B = 2`.

Gráfok megjelenítésére egy hasznos könyvtár a `graphviz`.

In [ ]:
import importlib
if importlib.util.find_spec('graphviz') is None:
    ! pip install graphviz
import graphviz

In [ ]:
edges = [[0,1], [1,2], [2,0]]
g0 = graphviz.Graph()

g0.edges([(str(a), str(b)) for a, b in edges])
g0

A gráf összefüggő, tetszőleges `A`, `B` esetén a válasz: `True`

In [ ]:
n = 6
edges = [[0,1],[0,2],[3,5],[5,4],[4,3]]
A = 0
B = 5

g1 = graphviz.Graph()
g1.edges([(str(a), str(b)) for a, b in edges])
g1

Nincs út 0 és 5 között. A válasz: `False`

Összefüggőségi komponenseket szeretnénk számolni.



## Ötlet.

Az él nélküli gráfból indulunk ki. Itt egy elemű komponensek vannak.

Minden komponensből válasszunk egy reprezentáns és minden $i$ pontra feljegyezzük, melyik komponensben van.




In [ ]:
def show_graph(roots, direction = 'LR'):
    g = graphviz.Digraph(graph_attr={'rankdir': direction})
    g.edges((str(i), str(r)) for i, r in enumerate(roots))
    return g

In [ ]:
n = 5
roots = [i for i in range(n)]
display(show_graph(roots, 'TD'))

ha behúzzuk a $(0, 1)$ élet, akkor $0$ és $1$ azonos komponensbe kerül. Választhatunk a két összeuniózott komponens reprezentánsa között, legyen pl. 1

In [ ]:
roots[0] = 1
display(show_graph(roots, 'TD'))

Ha most a (0, 2) élet akarjuk behúzni, akkor nem állíthatjuk át `roots[0]`. Meg kell keresnünk `0` komponensének reprezentását, ez 1 és vagy `roots[1]`-et állítjuk 2 -re, vagy `roots[2]`-t 1-re.

In [ ]:
def find(roots, a):
    while a != roots[a]:
        a = roots[a]
    return a

In [ ]:
find(roots, 0), find(roots, 2)

In [ ]:
def union(roots, a, b):
    ra = find(roots, a)
    rb = find(roots, b)
    roots[ra] = rb

In [ ]:
union(roots, 0, 2)
print(f"After union(0, 2) {roots=}")
display(show_graph(roots))

union(roots, 3, 4)
print(f"After union(3, 4) {roots=}")
print(roots)
display(show_graph(roots))

union(roots, 3, 2)
print(f"After union(3, 2) {roots=}")
print(roots)
display(show_graph(roots))

Ezután az a kérdés, hogy el lehet-e jutni `A`-ból, `B`-be könnyen eldönthető. Ha `A` és `B` azonos komponensben van, akkor `A` és `B` között megy út az eredeti gráfban, különben nem.

1. példa
`n = 3`, élek `edges = [[0,1], [1,2], [2,0]]`, `A =  0`, `B = 2`.

In [ ]:
def show_edges(edges, direction='LR'):
    g = graphviz.Graph(graph_attr={'rankdir': direction})
    g.edges([(str(a), str(b)) for a, b in edges])
    return g

In [ ]:
n = 3
edges = [[0,1], [1,2], [2,0]]
A =  0
B = 2

display(show_edges(edges, 'TD'))
print(f"Eredeti gráf")
print("="*50)

roots = [i for i in range(n)]
for a, b in edges:
    union(roots, a, b)

display(show_graph(roots))
print(f"{A=} és {B=} {'azonos' if find(roots, A)==find(roots, B) else 'különböző'} komponensben van")

2. példa

In [ ]:
n = 6
edges = [[0,1],[0,2],[3,5],[5,4],[4,3]]
A = 0
B = 5

display(show_edges(edges))
print(f"Eredeti gráf")
print("="*50)

roots = [i for i in range(n)]
for a, b in edges:
    union(roots, a, b)

display(show_graph(roots))
print(f"{A=} és {B=} {'azonos' if find(roots, A)==find(roots, B) else 'különböző'} komponensben van")


Mi történik, ha nagyobb gráfunk van?

In [ ]:
n = 10
edges = [(0,i+1) for i in range(n-1)]
display(show_edges(edges, 'TD'))

roots = [i for i in range(n)]
for a, b in edges:
    union(roots, a, b)

display(show_graph(roots))


Valahányszor behúzzuk a $(0, i)$ élet, meg kell keresni $0$ reprezentánsát. $k$ él behúzása után $k$-lépéssel találjuk meg. Ha 10 helyett 10_000 méretű a gráf ez nem fog működni.

### Javítási lehetőségek.

- Amikor megkeressük $i$ reprezentánsát végig megyünk a reprezentánshoz vezető úton. Minden meglátogatott csúcsra ismerté válik a reprezentáns értéke. Ezt beírhatjuk a `roots` tömbe. (path compression)

- A nagyobb komponensbe kössük be a kisebbet és ne fordítva. Ehhez  a ,,méretet'' nyilván kell tartani.

In [ ]:
def find_better(roots, a):
    ra = roots[a]
    if a != ra:
        ra = find_better(roots, ra)
        roots[a] = ra
    return ra

def find_better_without_recursion(roots, a):
    stack = []

    ra = roots[a]
    while a != ra:
        stack.append(a)
        a = ra
        ra = roots[a]

    while stack:
        roots[stack.pop()] = ra

    return ra


def union_sizes(roots, sizes, a, b):
    ra = find_better(roots, a)
    rb = find_better(roots, b)
    if ra != rb:
        if sizes[ra] < sizes[rb]:
            ra, rb = rb, ra
        roots[rb] = ra
        sizes[ra] += sizes[rb]

def union_ranks(roots, ranks, a, b):
    ra = find_better(roots, a)
    rb = find_better(roots, b)
    if ra != rb:
        if ranks[ra] < ranks[rb]:
            ra, rb = rb, ra
        roots[rb] = ra
        if ranks[ra] == ranks[rb]:
            ranks[ra] += 1




In [ ]:
n = 10
edges = [(0, i) for i in range(1, n)]
display(show_edges(edges, 'TD'))

roots = [i for i in range(n)]

for a, b in edges:
    ra = find_better(roots, a)
    rb = find_better(roots, b)
    roots[ra] = rb

display(show_graph(roots, 'LR'))


In [ ]:
n = 10
edges = [(0, i) for i in range(1, n)]
display(show_edges(edges, 'TD'))

roots = [i for i in range(n)]

for a, b in edges:
    ra = find_better(roots, a)
    rb = find_better(roots, b)
    roots[rb] = ra

display(show_graph(roots, 'TD'))


In [ ]:
n = 10
edges = [(0, i) for i in range(1, n)]
display(show_edges(edges, 'TD'))

roots = [i for i in range(n)]
sizes = [1]*n

for a, b in edges:
    union_sizes(roots, sizes, a, b)

display(show_graph(roots, 'TD'))


In [ ]:
n = 10
edges = [(0, i) for i in range(1, n)]
display(show_edges(edges, 'TD'))

roots = [i for i in range(n)]
ranks = [0]*n

for a, b in edges:
    union_ranks(roots, ranks, a, b)

display(show_graph(roots, 'TD'))


### Szokásos implementáció

In [ ]:
class UnionFind:
    def __init__(self, n):
        self.roots = [i for i in range(n)]
        self.sizes = [1]*n

    def find(self, a):
        ra = self.roots[a]
        if a != ra:
            ra = self.find( ra)
            self.roots[a] = ra
        return ra

    def union(self, a, b):
        ra = self.find(a)
        rb = self.find(b)
        if ra != rb:
            if self.sizes[ra] < self.sizes[rb]:
                ra, rb = rb, ra
            self.roots[rb] = ra
            self.sizes[ra] += self.sizes[rb]


In [ ]:
uf = UnionFind(10)
print(uf)
uf.union(1, 2)
print(uf)
uf.union(5, 9)
uf.union(6, 7)
print(uf)

uf

`__str__` és `__repr__` metódusok

In [ ]:
def as_set(seq):
    return f"{{{', '.join(map(str, seq))}}}"

def uf_str(self):
    components = {}
    for a in range(len(self.roots)):
        ra = self.find(a)
        if ra not in components:
            components[ra] = []
        components[ra].append(a)
    return f"{{{ ', '.join(map(as_set, components.values()))}}}"

def uf_repr(self):
    return f"{type(self).__name__}({len(self.roots)})"

# Így is lehet:
UnionFind.__str__ = uf_str
UnionFind.__repr__ = uf_repr

In [ ]:
uf = UnionFind(10)
print(uf)
uf.union(1, 2)
print(uf)
uf.union(5, 9)
uf.union(6, 7)
print(uf)

uf

### További kérdések

- Tegyük fel, hogy a komponensek száma érdekel minket. Hogyan oldanánk, meg, hogy konstans idő alatt megkaphassuk.
- Tegyük fel, hogy a legnagyobb komponens méretet érdekel minket. Hogyan oldanánk, meg, hogy konstans idő alatt megkaphassuk.
- Hogyan ellenőriznénk, hogy két partíció azonos?